In [1]:
# Imports

import json, yaml
import numpy as np
import pandas as pd
import datetime
import math
from sqlalchemy.engine.url import URL
from sqlalchemy import create_engine

import glob

In [2]:
# Open connections

config_file = '/Users/annaartigas/ada/TarifaJusta/TJ_config.yml'
with open(config_file, 'r') as conf:
    config = yaml.load(conf)

db_bi = config['db_bi']
conn_bi = create_engine(URL('mysql', **db_bi))

In [128]:
# Load files
SaludMarca_files =glob.glob("C:/Users/marta.cardona/Google Drive/03_Proyectos/02_OPE/OPE Data Team - Marta/Netquest Data/Salud de Marca_*.xlsx")
RecuerdoPublicitario_files =glob.glob("C:/Users/marta.cardona/Google Drive/03_Proyectos/02_OPE/OPE Data Team - Marta/Netquest Data/Recuerdo Publicitario_*.xlsx")
Medios_files =glob.glob("C:/Users/marta.cardona/Google Drive/03_Proyectos/02_OPE/OPE Data Team - Marta/Netquest Data/Medios_*.xlsx")
Categoria_files =glob.glob("C:/Users/marta.cardona/Google Drive/03_Proyectos/02_OPE/OPE Data Team - Marta/Netquest Data/Categoría_*.xlsx")

# Take last file (by date)
SaludMarca_last = max(SaludMarca_files)
RecuerdoPublicitario_last = max(RecuerdoPublicitario_files)
Medios_last = max(Medios_files)
Categoria_last = max(Categoria_files)

# Print names to check it
[SaludMarca_last, 
RecuerdoPublicitario_last,
Medios_last,
Categoria_last]

['C:/Users/marta.cardona/Google Drive/03_Proyectos/02_OPE/OPE Data Team - Marta/Netquest Data\\Salud de Marca_20200406.xlsx',
 'C:/Users/marta.cardona/Google Drive/03_Proyectos/02_OPE/OPE Data Team - Marta/Netquest Data\\Recuerdo Publicitario_20200406.xlsx',
 'C:/Users/marta.cardona/Google Drive/03_Proyectos/02_OPE/OPE Data Team - Marta/Netquest Data\\Medios_20200406.xlsx',
 'C:/Users/marta.cardona/Google Drive/03_Proyectos/02_OPE/OPE Data Team - Marta/Netquest Data\\Categoría_20200406.xlsx']

In [129]:
# Read excel sheets
TOM = pd.read_excel (SaludMarca_last, sheet_name='Slide_1', header=1)
Espontaneo = pd.read_excel (SaludMarca_last, sheet_name='Slide_2', header=1)
RecuerdoPubli = pd.read_excel (RecuerdoPublicitario_last, sheet_name='Slide_2', header=1)
MediosPubli = pd.read_excel (Medios_last, sheet_name='Slide_8', header=1)
ConsideraCambio = pd.read_excel (Categoria_last, sheet_name='Slide_8', header=1)
RecallConsideration = pd.read_excel (Categoria_last, sheet_name='Slide_12', header=1)

In [130]:
## TOM ##

# Fix columns
del TOM["Unnamed: 1"]
TOM.rename(columns={"Unnamed: 0" : "Date"}, inplace=True)

## Fix date format
# Get last part of the Date ( = sunday)
TOM['Date'] = TOM['Date'].str.strip().str[-8:]
TOM['Date']= pd.to_datetime(TOM['Date'], format='%d/%m/%y')
#TOM['Date_end'] = TOM['Date'] # just for testing

# Convert weekly to daily backwards. [Note: sunday's value represents previous weekdays values]
TOM = TOM.set_index('Date').resample('1D').backfill().reset_index()
#del TOM["Date_end"] # just for testing
TOM.head()

,Date,Endesa,Repsol,EDP,Iberdrola,Factor Energia,SomEnergia,Holaluz,Feníe,CEPSA,Naturgy
0,2018-12-09,0.467886,0.037882,0.029320,0.263788,0.006346,0.021487,0.050667,0.000764,0.005590,0.103429
1,2018-12-10,0.466214,0.033202,0.032192,0.262164,0.005109,0.026677,0.044478,0.001703,0.005129,0.101650
2,2018-12-11,0.466214,0.033202,0.032192,0.262164,0.005109,0.026677,0.044478,0.001703,0.005129,0.101650
3,2018-12-12,0.466214,0.033202,0.032192,0.262164,0.005109,0.026677,0.044478,0.001703,0.005129,0.101650
4,2018-12-13,0.466214,0.033202,0.032192,0.262164,0.005109,0.026677,0.044478,0.001703,0.005129,0.101650


In [131]:
## Espontaneo ##

# Fix columns
del Espontaneo["Unnamed: 1"]
Espontaneo.rename(columns={"Unnamed: 0" : "Date"}, inplace=True)

## Fix date format
# Get last part of the Date ( = sunday)
Espontaneo['Date'] = Espontaneo['Date'].str.strip().str[-8:]
Espontaneo['Date']= pd.to_datetime(Espontaneo['Date'], format='%d/%m/%y')
#Espontaneo['Date_end'] = Espontaneo['Date'] # just for testing

# Convert weekly to daily backwards. [Note: sunday's value represents previous weekdays values]
Espontaneo = Espontaneo.set_index('Date').resample('1D').backfill().reset_index()
#del Espontaneo["Date_end"] # just for testing
Espontaneo.head()

,Date,Endesa,Repsol,EDP,Iberdrola,Factor Energia,SomEnergia,Holaluz,Feníe,CEPSA,Naturgy
0,2018-12-09,0.770607,0.196159,0.151588,0.688617,0.062357,0.063167,0.194052,0.007703,0.055225,0.435416
1,2018-12-10,0.764721,0.170190,0.140556,0.678632,0.060719,0.060738,0.173184,0.011689,0.047314,0.445678
2,2018-12-11,0.764721,0.170190,0.140556,0.678632,0.060719,0.060738,0.173184,0.011689,0.047314,0.445678
3,2018-12-12,0.764721,0.170190,0.140556,0.678632,0.060719,0.060738,0.173184,0.011689,0.047314,0.445678
4,2018-12-13,0.764721,0.170190,0.140556,0.678632,0.060719,0.060738,0.173184,0.011689,0.047314,0.445678


In [132]:
## Recuerdo Publicitario ##

# Fix columns
del RecuerdoPubli["Unnamed: 1"]
RecuerdoPubli.rename(columns={"Unnamed: 0" : "Date"}, inplace=True)

## Fix date format
# Get last part of the Date ( = sunday)
RecuerdoPubli['Date'] = RecuerdoPubli['Date'].str.strip().str[-8:]
RecuerdoPubli['Date']= pd.to_datetime(RecuerdoPubli['Date'], format='%d/%m/%y')
#RecuerdoPubli['Date_end'] = RecuerdoPubli['Date'] # just for testing

# Convert weekly to daily backwards. [Note: sunday's value represents previous weekdays values]
RecuerdoPubli = RecuerdoPubli.set_index('Date').resample('1D').backfill().reset_index()
#del RecuerdoPubli["Date_end"] # just for testing
RecuerdoPubli.head()

,Date,Endesa,Repsol,EDP,Iberdrola,Factor Energia,SomEnergia,Holaluz,Feníe,CEPSA,Naturgy
0,2018-12-09,0.332651,0.110308,0.062777,0.340711,0.038735,0.023873,0.098716,0.000792,0.048871,0.165485
1,2018-12-10,0.342421,0.099626,0.052910,0.334902,0.037665,0.027471,0.090460,0.001702,0.040627,0.170182
2,2018-12-11,0.342421,0.099626,0.052910,0.334902,0.037665,0.027471,0.090460,0.001702,0.040627,0.170182
3,2018-12-12,0.342421,0.099626,0.052910,0.334902,0.037665,0.027471,0.090460,0.001702,0.040627,0.170182
4,2018-12-13,0.342421,0.099626,0.052910,0.334902,0.037665,0.027471,0.090460,0.001702,0.040627,0.170182


In [133]:
## Medios Publicitarios ##

# Delete first row
MediosPubli = MediosPubli[MediosPubli.Youtube != 'Holaluz']
MediosPubli.rename(columns={"Unnamed: 0" : "Date"}, inplace=True)
MediosPubli = MediosPubli[MediosPubli.Date != 'Recuento']

## Fix date format
# Get last part of the Date ( = sunday)
MediosPubli['Date'] = MediosPubli['Date'].str.strip().str[-8:]
# Delete blank rows and last "Recuento" row
MediosPubli = MediosPubli[MediosPubli['Televisión'].notna()]
MediosPubli = MediosPubli[MediosPubli['Date'].notna()]
# Convert to date format
MediosPubli['Date']= pd.to_datetime(MediosPubli['Date'], format='%d/%m/%y')
#MediosPubli['Date_end'] = MediosPubli['Date'] # just for testing
# Convert weekly to daily backwards. [Note: sunday's value represents previous weekdays values]
MediosPubli = MediosPubli.set_index('Date').resample('1D').backfill().reset_index()
#del MediosPubli["Date_end"] # just for testing
MediosPubli.head()

,Date,Televisión,Prensa/ revistas,Lonas en Edificios,En Autobuses,En Metro,En repartidores de prensa gratuita,Radio,Internet: Publicidad,Redes Sociales,Youtube,Mail,Noticias hablando de la compañía,Conversaciones con Amigos familiares o compañeros,Ninguno/ no lo recuerdo
0,2018-12-23,0.416823,0.0831741,0,0.0415808,0.0206659,0.0415808,0.104332,0.353842,0.166578,0.0415871,0.0834231,0.062751,0.104076,0.0627447
1,2018-12-24,0.48889,0.0817212,0.00969729,0.0406341,0.0204216,0.0402997,0.121896,0.316225,0.142936,0.0209083,0.0913203,0.0616437,0.112181,0.0516088
2,2018-12-25,0.48889,0.0817212,0.00969729,0.0406341,0.0204216,0.0402997,0.121896,0.316225,0.142936,0.0209083,0.0913203,0.0616437,0.112181,0.0516088
3,2018-12-26,0.48889,0.0817212,0.00969729,0.0406341,0.0204216,0.0402997,0.121896,0.316225,0.142936,0.0209083,0.0913203,0.0616437,0.112181,0.0516088
4,2018-12-27,0.48889,0.0817212,0.00969729,0.0406341,0.0204216,0.0402997,0.121896,0.316225,0.142936,0.0209083,0.0913203,0.0616437,0.112181,0.0516088


In [134]:
## CONSIDERACION CAMBIO DE COMPAÑIA ##

# Fix columns
del ConsideraCambio["Unnamed: 1"]
ConsideraCambio.rename(columns={"Unnamed: 0" : "Date"}, inplace=True)
ConsideraCambio.rename(columns={"Sí, estoy pensando en cambiar de compañía" : "Considera_Cambio_Compañia"}, inplace=True)

# Delete blank rows and last "Recuento" row
ConsideraCambio = ConsideraCambio[ConsideraCambio['Considera_Cambio_Compañia'].notna()]
ConsideraCambio = ConsideraCambio[ConsideraCambio['Date'].notna()]

## Fix date format
# Get last part of the Date ( = sunday)
ConsideraCambio['Date'] = ConsideraCambio['Date'].str.strip().str[-8:]
ConsideraCambio['Date']= pd.to_datetime(ConsideraCambio['Date'], format='%d/%m/%y')
#ConsideraCambio['Date_end'] = ConsideraCambio['Date'] # just for testing

# Convert weekly to daily backwards. [Note: sunday's value represents previous weekdays values]
ConsideraCambio = ConsideraCambio.set_index('Date').resample('1D').backfill().reset_index()
#del ConsideraCambio["Date_end"] # just for testing
ConsideraCambio.head()

,Date,Considera_Cambio_Compañia
0,2018-12-09,0.279386
1,2018-12-10,0.298426
2,2018-12-11,0.298426
3,2018-12-12,0.298426
4,2018-12-13,0.298426


In [141]:
## Recall & Consideration ##
RecallConsideration = pd.read_excel (Categoria_last, sheet_name='Slide_12', header=1)

# Fix columns
del RecallConsideration["Unnamed: 1"]
RecallConsideration.rename(columns={"Unnamed: 0" : "Date"}, inplace=True)
RecallConsideration.rename(columns={"Endesa" : "Recall_Consideration"}, inplace=True)

# Look for the row where "Holaluz" values start
# Get row index where a new company values start:
company_names_index = RecallConsideration[RecallConsideration['Date'].isna()].index
# Start Index
start_index = company_names_index[RecallConsideration.iloc[company_names_index,]['Recall_Consideration']=='Holaluz']+1 # 497 -> Start
#start_index = RecallConsideration[(RecallConsideration.Date.isna()) & (RecallConsideration.Recall_Consideration =='Holaluz')]
start_index = start_index[0]
# Select from start_index
RecallConsideration = RecallConsideration.iloc[start_index:,].reset_index()
# End Index
end_index = RecallConsideration[RecallConsideration['Date'].isna()].index[0]
# Select until end_date
RecallConsideration = RecallConsideration.iloc[:end_index,].reset_index()
# Delete old index columns
del RecallConsideration["level_0"]
del RecallConsideration["index"]

## Fix date format
# Get last part of the Date ( = sunday)
RecallConsideration['Date'] = RecallConsideration['Date'].str.strip().str[-8:]
RecallConsideration['Date']= pd.to_datetime(RecallConsideration['Date'], format='%d/%m/%y')
#RecallConsideration['Date_end'] = RecallConsideration['Date'] # just for testing

# Convert weekly to daily backwards. [Note: sunday's value represents previous weekdays values]
RecallConsideration = RecallConsideration.set_index('Date').resample('1D').backfill().reset_index()
#del RecallConsideration["Date_end"] # just for testing
RecallConsideration.head()

,Date,Recall_Consideration
0,2018-12-09,54.5834
1,2018-12-10,53.5369
2,2018-12-11,53.5369
3,2018-12-12,53.5369
4,2018-12-13,53.5369


In [ ]:
# TOM - Update DB #

# Load Dates from DB
TOM_old_dates =  pd.read_sql("""
    SELECT Date FROM OPE_TOM
    """, conn_bi)
TOM_old_dates.head()

# Select only new dates
TOM_new =TOM[~TOM.Date.isin(TOM_old_dates.Date)]
print("TOM - New rows:", len(TOM_new))

# Upload new data to DB
TOM_new.to_sql(name='OPE_TOM', con=conn_bi, if_exists="append", index=False)


In [155]:
# ESPONTANEO - Update DB #

# Load Dates from DB
Espontaneo_old_dates =  pd.read_sql("""
    SELECT Date FROM OPE_ESPONTANEO
    """, conn_bi)
Espontaneo_old_dates.head()

# Select only new dates
Espontaneo_new =Espontaneo[~Espontaneo.Date.isin(Espontaneo_old_dates.Date)]
print("Espontaneo - New rows:", len(Espontaneo_new))

# Upload new data to DB
Espontaneo_new.to_sql(name='OPE_ESPONTANEO', con=conn_bi, if_exists="append", index=False)

In [156]:
# RecuerdoPubli - Update DB #

# Load Dates from DB
RecuerdoPubli_old_dates =  pd.read_sql("""
    SELECT Date FROM OPE_RECUERDO_PUBLI
    """, conn_bi)
RecuerdoPubli_old_dates.head()

# Select only new dates
RecuerdoPubli_new =RecuerdoPubli[~RecuerdoPubli.Date.isin(RecuerdoPubli_old_dates.Date)]
print("Recuerdo Publicitario - New rows:", len(RecuerdoPubli_new))

# Upload new data to DB
RecuerdoPubli_new.to_sql(name='OPE_RECUERDO_PUBLI', con=conn_bi, if_exists="append", index=False)



In [157]:
# MediosPubli - Update DB # 

# Load Dates from DB
MediosPubli_old_dates =  pd.read_sql("""
    SELECT Date FROM OPE_MEDIOS_PUBLI
    """, conn_bi)
MediosPubli_old_dates.head()

# Select only new dates
MediosPubli_new =MediosPubli[~MediosPubli.Date.isin(MediosPubli_old_dates.Date)]
print("MediosPubli - New rows:", len(MediosPubli_new))

# Upload new data to DB
MediosPubli_new.to_sql(name='OPE_MEDIOS_PUBLI', con=conn_bi, if_exists="append", index=False)


In [158]:
# ConsideraCambio - Update DB #

# Load Dates from DB
ConsideraCambio_old_dates =  pd.read_sql("""
    SELECT Date FROM OPE_CONSIDERA_CAMBIO
    """, conn_bi)
ConsideraCambio_old_dates.head()

# Select only new dates
ConsideraCambio_new =ConsideraCambio[~ConsideraCambio.Date.isin(ConsideraCambio_old_dates.Date)]
print("Considera Cambio Compañia - New rows:", len(ConsideraCambio_new))

# Upload new data to DB
ConsideraCambio_new.to_sql(name='OPE_CONSIDERA_CAMBIO', con=conn_bi, if_exists="append", index=False)


In [159]:
# RecallConsideration - Update DB #

# only first tome:
RecallConsideration.to_sql(name='OPE_RECALL_CONSIDERATION', con=conn_bi, if_exists="append", index=False)

# Load Dates from DB
RecallConsideration_old_dates =  pd.read_sql("""
    SELECT Date FROM OPE_RECALL_CONSIDERATION
    """, conn_bi)
RecallConsideration_old_dates.head()

# Select only new dates
RecallConsideration_new =RecallConsideration[~RecallConsideration.Date.isin(RecallConsideration_old_dates.Date)]
print("RecallConsideration - New rows:", len(RecallConsideration_new))

# Upload new data to DB
RecallConsideration_new.to_sql(name='OPE_RECALL_CONSIDERATION', con=conn_bi, if_exists="append", index=False)


In [ ]:
## ------ DB Actions ------ ##
# Check
#already_in =  pd.read_sql("""
#    SELECT * FROM OPE_TOM
#    """, conn_bi)
#already_in.head()

# Delete table
#pd.read_sql("""
#    DROP TABLE OPE_TOM
#    """, conn_bi)